# semantically searchable multilingual transcription pipeline

This document details a modular pipeline that takes in an audio/video file, transcribes it, translates the transcription into a desired language, and makes the result semantically searchable.

To follow along with this demonstration be sure to initialize your krixik session with your api key and url as shown below. 

We illustrate loading these required secrets in via [python-dotenv](https://pypi.org/project/python-dotenv/), storing those secrets in a `.env` file.  This is always good practice for storing / loading secrets (e.g., doing so will reduce the chance you inadvertantly push secrets to a repo).


In [1]:
import sys 
sys.path.append('../../../')

from docs.utilities.reset import reset_pipeline

In [2]:
# load secrets from a .env file using python-dotenv
from dotenv import load_dotenv
import os
load_dotenv("../../.env")
MY_API_KEY = os.getenv('MY_API_KEY')
MY_API_URL = os.getenv('MY_API_URL')

# import krixik and initialize it with your personal secrets
from krixik import krixik
krixik.init(api_key = MY_API_KEY, 
            api_url = MY_API_URL)

SUCCESS: You are now authenticated.


This small function prints dictionaries very nicely in notebooks / markdown.

In [3]:
# print dictionaries / json nicely in notebooks / markdown
import json
def json_print(data):
    print(json.dumps(data, indent=2))

A table of contents for the remainder of this document is shown below.


- [pipeline setup](#pipeline-setup)
- [processing a file](#processing-a-file)
- [performing semantic search](#performing-semantic-search)
- [saving the pipeline config for future use](#saving-the-pipeline-config-for-future-use)

## pipeline setup

Below we setup a multi module pipeline to serve our intended purpose, which is to build a pipeline that will transcribe any audio/video and make it semantically searchable in any language.

To do this we will use the following modules:

- [`transcribe`](modules/transcribe.md): takes in audio/video input, outputs json of content transcription
- [`translate`](modules/translate.md): takes in json of text snippets, outputs json of translated snippets
- [`json-to-txt`](modules/json-to-txt.md): takes in json of text snippets, merges into text file
- [`parser`](modules/parser.md): takes in text, slices into (possibly overlapping) strings
- [`text-embedder`](modules/text-embedder.md): takes in text snippets, creates vector representation of each outputing an npy file
- [`vector-db`](modules/vector-db.md): takes in npy of vectors, outputs vector db

In [4]:
from krixik.pipeline_builder.module import Module
from krixik.pipeline_builder.pipeline import CreatePipeline

# select modules
module_1 = Module(module_type="transcribe")
module_2 = Module(module_type="translate")
module_3 = Module(module_type="json-to-txt")
module_4 = Module(module_type="parser")
module_5 = Module(module_type="text-embedder")
module_6 = Module(module_type="vector-db")

# create custom pipeline object
custom = CreatePipeline(name='transcribe-translate-semantic-pipeline', 
                        module_chain=[module_1, module_2, module_3, module_4, module_5, module_6])

# pass the custom object to the krixik operator (note you can also do this by passing its config)
pipeline = krixik.load_pipeline(pipeline=custom)

With our `custom` pipeline built we now pass it, along with a test file, to our operator to process the file.

## processing a file

We first define a path to a local input file.

In [5]:
# define path to an input file
test_file = "../../input_data/Interesting Facts About Colombia.mp4"

Lets take a quick look at this file before processing.

In [6]:
# examine contents of input file
from IPython.display import Video
Video(test_file)

The input video content language content is English.  We will use the `opus-mt-en-es` model of the [`translate`](modules/translate.md) to translate the transcript of this video into Spanish.

For this run we will use the default models for the remainder of the modules.


In [7]:
# delete all processed datapoints belonging to this pipeline
reset_pipeline(pipeline)

In [8]:
# test file
test_file = "../../input_data/Interesting Facts About Colombia.mp4"

# process test input
process_output = pipeline.process(local_file_path = test_file,
                                  expire_time=60*5,
                                  modules={"translate": {"model": "opus-mt-en-es"}})

INFO: Checking that file size falls within acceptable parameters...
INFO:...success!
converted ../../input_data/Interesting Facts About Colombia.mp4 to: /var/folders/k9/0vtmhf0s5h56gt15mkf07b1r0000gn/T/tmps__us5t0/krixik_converted_version_Interesting Facts About Colombia.mp3
INFO: hydrated input modules: {'transcribe': {'model': 'whisper-tiny', 'params': {}}, 'translate': {'model': 'opus-mt-en-es', 'params': {}}, 'json-to-txt': {'model': 'base', 'params': {}}, 'parser': {'model': 'sentence', 'params': {}}, 'text-embedder': {'model': 'multi-qa-MiniLM-L6-cos-v1', 'params': {'quantize': True}}, 'vector-db': {'model': 'faiss', 'params': {}}}
INFO: symbolic_directory_path was not set by user - setting to default of /etc
INFO: file_name was not set by user - setting to random file name: krixik_generated_file_name_oogbwiauhc.mp3
INFO: wait_for_process is set to True.
INFO: file will expire and be removed from you account in 300 seconds, at Mon Apr 29 18:06:15 2024 UTC
INFO: transcribe-transla

The output of this process is printed below.  Because the output of this particular pipeline is a database file, the process output is shown as null in the output.  The local address of the output file itself has been returned to the address noted in the `process_output_files` key.

In [9]:
# nicely print the output of this process
json_print(process_output)

{
  "status_code": 200,
  "pipeline": "transcribe-translate-semantic-pipeline",
  "request_id": "b4e43bd1-bb20-4872-b93e-8f640fb34193",
  "file_id": "2926aa4e-5c01-4e49-9e83-5eb10a926ef1",
  "message": "SUCCESS - output fetched for file_id 2926aa4e-5c01-4e49-9e83-5eb10a926ef1.Output saved to location(s) listed in process_output_files.",
  "warnings": [],
  "process_output": null,
  "process_output_files": [
    "/Users/jeremywatt/Desktop/krixik-cli/docs/examples/transcribe/2926aa4e-5c01-4e49-9e83-5eb10a926ef1.faiss"
  ]
}


## performing semantic search

Because our pipeline has `text-embedder` and `vector-db` modules we can semantically search the translated transcription, here in Spanish (since we processed our file with an English-Spanish model).  

In [10]:
# semantically search translated transcription
search_output = pipeline.semantic_search(query="hechos realmente bsicos", 
                                         file_ids=[process_output["file_id"]])

json_print(search_output)

{
  "status_code": 200,
  "request_id": "3b4f827c-e685-4157-b445-6c79c1ad7ab3",
  "message": "Successfully queried 1 user file.",
  "warnings": [],
  "items": [
    {
      "file_id": "2926aa4e-5c01-4e49-9e83-5eb10a926ef1",
      "file_metadata": {
        "file_name": "krixik_generated_file_name_oogbwiauhc.mp3",
        "symbolic_directory_path": "/etc",
        "file_tags": [],
        "num_vectors": 39,
        "created_at": "2024-04-30 01:01:18",
        "last_updated": "2024-04-30 01:01:18"
      },
      "search_results": [
        {
          "snippet": "Miramos algunos hechos realmente bsicos.",
          "line_numbers": [
            1
          ],
          "distance": 0.093
        },
        {
          "snippet": "Bienvenidos de nuevo a los hechos F2D.",
          "line_numbers": [
            1
          ],
          "distance": 0.267
        },
        {
          "snippet": "Es interesante el hecho de que en 2007, los principales lugares que equivalan a una zona de amor

Learn more about the [`semantic_search` method here](system/semantic_search.md).

## saving the pipeline config for future use

You can save the configuration of this pipeline using the `custom` object, and use it later direclty without building it again in python.

In [11]:
# save your config for later use (that way you don't need to re-build in python)
custom.save(config_path='transcribe-translate-semantic-pipeline.yml')

See more about [saving and loading pipeline configuration files](LINNK GOES HERE).